In [1]:
import csv 
import pandas as pd
from bokeh.plotting import figure, show
from datetime import datetime

In [2]:
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from math import pi
from bokeh.models import LabelSet, ColumnDataSource

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
nt_df=pd.read_csv('D:/DATASETS/NETFLIX/netflix_titles.csv')

In [4]:
nt_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
nt_df['date_added']=pd.to_datetime(nt_df['date_added'])

In [6]:
nt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       7787 non-null   object        
 1   type          7787 non-null   object        
 2   title         7787 non-null   object        
 3   director      5398 non-null   object        
 4   cast          7069 non-null   object        
 5   country       7280 non-null   object        
 6   date_added    7777 non-null   datetime64[ns]
 7   release_year  7787 non-null   int64         
 8   rating        7780 non-null   object        
 9   duration      7787 non-null   object        
 10  listed_in     7787 non-null   object        
 11  description   7787 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 730.2+ KB


In [7]:
data= pd.Series(nt_df['type']).value_counts().reset_index(name='value').rename(columns={'index':'Type'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
colors = ["red", "green", "blue", "orange", "yellow"]
data['color'] = colors[:len(data)]
data['percentage'] =round(data['value'] / data['value'].sum() * 100,2)

In [8]:
data

,Type,value,angle,color,percentage
0,Movie,5377,4.338601,red,69.05
1,TV Show,2410,1.944584,green,30.95


In [9]:
p = figure(plot_height=350, title="CONTENT DISTRIBUTION", toolbar_location=None,
           tools="hover", tooltips="@Type: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white",  legend_field='Type', source=data,color='color')

data["value"] = data['value'].astype(str)
data["value"] = data["value"].str.pad(35, side = "left")
source = ColumnDataSource(data)

labels = LabelSet(x=0, y=1, text='value',
        angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

In [ ]:
data= pd.Series(nt_df['country']).value_counts().reset_index(name='value').rename(columns={'index':'Country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
colors = ["red", "green", "blue", "orange", "yellow"]
data=data.head(15)
data['color'] = Category20c[len(data)]
data['percentage'] =round(data['value'] / data['value'].sum() * 100,2)

In [ ]:
p = figure(plot_height=500, title="TOP 15 COUNTRIES CONTRIBUTION", toolbar_location=None,
           tools="hover", tooltips="@Type: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white",  legend_field='Country', source=data,color='color')

data["percentage"] = data['percentage'].astype(str)
data["percentage"] = data["percentage"].str.pad(35, side = "left")
source = ColumnDataSource(data)

labels = LabelSet(x=0, y=1, text='percentage',
        angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

In [ ]:
source =ColumnDataSource(dict(x=data['Country'],y=data['value']))
p = figure(x_range=data['Country'], plot_height=400, title="TOP 15 COUNTRIES CONTENT",plot_width=900,
           toolbar_location=None, tools="")

p.vbar(x=data['Country'], top=data['value'], width=0.8)
labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-155, y_offset=0, source=source, render_mode='canvas')


p.add_layout(labels)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
tv_df=nt_df[nt_df['type']=='TV Show']
tv_df['year']=tv_df['date_added'].dt.year
tv_df.head()

movie_df=nt_df[nt_df['type']=='Movie']
movie_df['year']=movie_df['date_added'].dt.year
movie_df.head()

nt_df['year']=nt_df['date_added'].dt.year


In [ ]:
data= pd.Series(tv_df['year']).value_counts().reset_index(name='value').rename(columns={'index':'Year'})
colors = ["red", "green", "blue", "orange", "yellow"]
data_m= pd.Series(movie_df['year']).value_counts().reset_index(name='value').rename(columns={'index':'Year'})
data_t= pd.Series(nt_df['year']).value_counts().reset_index(name='value').rename(columns={'index':'Year'})

In [ ]:
data=data.sort_values(by='Year')
data_m=data_m.sort_values(by='Year')
data_t=data_t.sort_values(by='Year')
data.head()

In [ ]:
p = figure(plot_height=400, title="CONTENT OVER THE YEARS",plot_width=900,
           toolbar_location=None, tools="")
source=ColumnDataSource(data)
source_m=ColumnDataSource(data_m)
source_t=ColumnDataSource(data_t)

p.line(x='Year', y='value', line_width=2, source=source,color='red',legend_label='TV Show')
p.line(x='Year', y='value', line_width=2, source=source_m,color='green',legend_label='Movie')
p.line(x='Year', y='value', line_width=2, source=source_t,color='blue',legend_label='All')

p.legend.location = "top_left"
p.legend.click_policy="hide"

show(p)

In [ ]:
tv_df.head()

data= pd.Series(tv_df['duration']).value_counts().reset_index(name='value').rename(columns={'index':'Country'})
#colors = ["red", "green", "blue", "orange", "yellow"]
data['color'] = Category20c[len(data)]


In [ ]:
source =ColumnDataSource(dict(x=data['Country'],y=data['value']))
p = figure(x_range=data['Country'], plot_height=400, title="TV SHOW SEASONS",plot_width=1000,
           toolbar_location=None, tools="")

p.vbar(x=data['Country'], top=data['value'], width=0.8,bottom=0)
labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-13.5, y_offset=0, source=source, render_mode='canvas')


p.add_layout(labels)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
data= pd.Series(movie_df['duration']).value_counts().reset_index(name='value').rename(columns={'index':'Country'})
#colors = ["red", "green", "blue", "orange", "yellow"]
#data['color'] = Category20c[len(data)]


In [ ]:
data=data.head(30)
source =ColumnDataSource(dict(x=data['Country'],y=data['value']))
p = figure(x_range=data['Country'], plot_height=400, title="MOVIE DURATION",plot_width=1000,
           toolbar_location=None, tools="")

p.vbar(x=data['Country'], top=data['value'], width=0.4,bottom=0)
labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-13.5, y_offset=0, source=source, render_mode='canvas')


p.add_layout(labels)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
# Genre of the tv shows listed
tv_df['genre']=tv_df['listed_in'].str.split(',')
g=[]
for i in tv_df['genre']:
    for j in i:
        w=j.strip()
        g.append(w)

In [ ]:
#Getting unique genres and count of the genres
from collections import Counter
c=Counter(g)
print ((c))

In [ ]:
df = pd.DataFrame.from_dict(c, orient='index').reset_index()
df=df.rename(columns={'index':'Genre',0:'Value'})
df.head()
df=df.sort_values(by=['Value'])
df=df.head(10)

In [ ]:
source =ColumnDataSource(dict(dict(x=df['Genre'],y=df['Value'])))
p = figure(x_range=df['Genre'], plot_height=400, title="TOP TV SHOW GENRES",plot_width=1000,
           toolbar_location=None, tools="")

p.vbar(x=df['Genre'], top=df['Value'], width=0.4,bottom=0)
labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-13.5, y_offset=0, source=source, render_mode='canvas')


p.add_layout(labels)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
#GENRES OF MOVIES
movie_df['genre']=movie_df['listed_in'].str.split(',')
g=[]
for i in movie_df['genre']:
    for j in i:
        w=j.strip()
        g.append(w)
        
from collections import Counter
c=Counter(g)
print ((c))

In [ ]:
df = pd.DataFrame.from_dict(c, orient='index').reset_index()
df=df.rename(columns={'index':'Genre',0:'Value'})
df.head()
df=df.sort_values(by=['Value'])
df=df.head(10)

In [ ]:
source =ColumnDataSource(dict(dict(x=df['Genre'],y=df['Value'])))
p = figure(x_range=df['Genre'], plot_height=400, title="TOP 10 MOVIE GENRES",plot_width=1000,
           toolbar_location=None, tools="")

p.vbar(x=df['Genre'], top=df['Value'], width=0.4,bottom=0)
labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-13.5, y_offset=0, source=source, render_mode='canvas')


p.add_layout(labels)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [ ]:
nt_df.head()